# Waitoa Fusion Audience

In [1]:
# Setup the environment



In [3]:
#pip install geopandas

In [5]:
#pip install matplotlib


#### ### Create pipeline to read all Dunnhumby data for Waitoa


In [91]:
### Create pipeline to read all Waitoa Dunnhumby data
import pandas as pd

# Read the Waitoa Dunnhumby frozen data
waitoa_frozen = pd.read_csv(r'G:\My Drive\together_projects\waitoa\fusion/together-dunhumbby-frozen-data.csv')

waitoa_frozen.head(2)

,AGG_TYPE,PRODUCT_GROUP,PRODUCT_NAME,PRODUCT_CODE,STORE_CLUSTER,FORMAT_NAME,STORE_REGION,STORE_NAME,STORE_CODE,09/09/2024_SALES,...,30/09/2024_VOLUME,07/10/2024_VOLUME,14/10/2024_VOLUME,21/10/2024_VOLUME,28/10/2024_VOLUME,04/11/2024_VOLUME,11/11/2024_VOLUME,18/11/2024_VOLUME,25/11/2024_VOLUME,02/12/2024_VOLUME
0,WEEKLY,13006 - Poultry - Frozen Foods,BOSTOCK BROTHERS CHICKEN TENDERS 400G EA 1,5321336_EA1,PNS Local Stop,PAKnSAVE,CENTRE,Napier City PAKnSAVE,8056_5000,177.48,...,7.0,13.0,7.0,8.0,3.0,1.0,NaN,NaN,NaN,NaN
1,WEEKLY,13006 - Poultry - Frozen Foods,TEGEL T/O FR LOUSIANA STYLE TENDERS 500G EA 1,5261498_EA1,PNS Local Stop,PAKnSAVE,SOUTH,PAK'nSAVE Dunedin,500007_12,3524.49,...,106.0,93.0,131.0,121.0,170.0,127.0,108.0,233.0,103.0,293.0


In [ ]:
# Get all columns that contain '_SALES' in their name
sales_column = [col for col in waitoa_frozen.columns if '_SALES' in col]

# Sum all sales columns to create total_sales
waitoa_frozen['total_sales'] = waitoa_frozen[sales_column].sum(axis=1)
# Drop all individual sales columns
waitoa_frozen = waitoa_frozen.drop(columns=sales_column)

# Display the first few rows to verify
#waitoa_frozen[['PRODUCT_GROUP', 'total_sales'] + sales_columns].head()
waitoa_frozen


## Do same for Volume
# Get all columns that contain '_SALES' in their name
volume_column = [col for col in waitoa_frozen.columns if '_VOLUME' in col]

# Sum all sales columns to create total_sales
waitoa_frozen['sales_volume'] = waitoa_frozen[volume_column].sum(axis=1)

# Drop all individual sales columns
waitoa_frozen = waitoa_frozen.drop(columns=volume_column)

# Drop specified columns
columns_to_drop = ['AGG_TYPE', 'PRODUCT_GROUP', 'PRODUCT_CODE', 'STORE_CLUSTER','STORE_REGION']
waitoa_frozen = waitoa_frozen.drop(columns=columns_to_drop)

# Display first few rows to verify
#waitoa_frozen.head()

In [9]:
## In the Product_name column, separate Waitoa from the rest of competitors

# Remove records containing "INGHAMS" in PRODUCT_NAME so we can separate Waitoa from the rest of competitors
waitoa_frozen = waitoa_frozen[~waitoa_frozen['PRODUCT_NAME'].str.contains('INGHAMS', case=True, na=False)]

# Display first few rows to verify
waitoa_frozen.head()

,PRODUCT_NAME,FORMAT_NAME,STORE_NAME,STORE_CODE,total_sales,sales_volume
0,BOSTOCK BROTHERS CHICKEN TENDERS 400G EA 1,PAKnSAVE,Napier City PAKnSAVE,8056_5000,995.87,73.0
1,TEGEL T/O FR LOUSIANA STYLE TENDERS 500G EA 1,PAKnSAVE,PAK'nSAVE Dunedin,500007_12,26409.61,2027.0
2,TEGEL FRZ FR TEMPURA BATTRD NUGGETS 900G EA 1,New World,New World Blenheim,500079_11,3224.10,222.0
4,INGHAM FRZN CHICKEN TENDERLOIN 400G EA 1,New World,Opotiki New World,3118_4000,1708.05,145.0
5,TEGEL FROZ CHKN NIBBLES 1.5KG EA 1,PAKnSAVE,Warkworth PAKnSAVE,8283_5000,1601.11,89.0


In [10]:
# Create the is_waitoa column
waitoa_frozen['is_waitoa'] = waitoa_frozen['PRODUCT_NAME'].str.contains('WAITOA', case=False, na=False)

# Create the pivot table
pivot_frozen = waitoa_frozen.pivot_table(
    index=['STORE_NAME', 'FORMAT_NAME'],
    values=['total_sales', 'sales_volume'],
    columns='is_waitoa',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Flatten the MultiIndex columns - modified to handle index columns correctly
pivot_frozen.columns = [
    col[0] if col[1] == '' else  # For index columns
    f'{col[0]}_{"waitoa" if col[1] else "competitors"}'  # For value columns
    for col in pivot_frozen.columns
]

# Reorder columns
column_order = [
    'STORE_NAME',
    'FORMAT_NAME',
    'total_sales_waitoa',
    'sales_volume_waitoa',
    'total_sales_competitors',
    'sales_volume_competitors'
]
pivot_frozen = pivot_frozen[column_order]

# Display the first few rows
pivot_frozen.head()

,STORE_NAME,FORMAT_NAME,total_sales_waitoa,sales_volume_waitoa,total_sales_competitors,sales_volume_competitors
0,Albany New World,New World,45151.04,4010.0,72041.99,5521.0
1,Albany PAKnSAVE,PAKnSAVE,110547.06,10494.0,330216.61,26759.0
2,Alderman Drive Henderson PAKnSAVE,PAKnSAVE,17440.96,1704.0,208490.36,15966.0
3,Aokautere New World,New World,13025.54,1152.0,34871.74,2637.0
4,Birkenhead New World,New World,52452.90,4624.0,84119.40,6491.0


In [11]:
# Calculate share of sales and volume
pivot_frozen['share_of_sales'] = pivot_frozen['total_sales_waitoa'] / pivot_frozen['total_sales_competitors']
pivot_frozen['share_of_volume'] = pivot_frozen['sales_volume_waitoa'] / pivot_frozen['sales_volume_competitors']

# Display first few rows to verify
pivot_frozen.head()

,STORE_NAME,FORMAT_NAME,total_sales_waitoa,sales_volume_waitoa,total_sales_competitors,sales_volume_competitors,share_of_sales,share_of_volume
0,Albany New World,New World,45151.04,4010.0,72041.99,5521.0,0.626732,0.726318
1,Albany PAKnSAVE,PAKnSAVE,110547.06,10494.0,330216.61,26759.0,0.334771,0.392167
2,Alderman Drive Henderson PAKnSAVE,PAKnSAVE,17440.96,1704.0,208490.36,15966.0,0.083654,0.106727
3,Aokautere New World,New World,13025.54,1152.0,34871.74,2637.0,0.373527,0.436860
4,Birkenhead New World,New World,52452.90,4624.0,84119.40,6491.0,0.623553,0.712371


In [12]:

pivot_frozen_df = pivot_frozen[['STORE_NAME', 'share_of_sales', 'share_of_volume']]

In [92]:
## Do same for Waito Deli
waitoa_deli = pd.read_csv(r'G:\My Drive\together_projects\waitoa\fusion/together-dunhumbby-deli-data.csv')
waitoa_deli.head(2)

,AGG_TYPE,PRODUCT_GROUP,PRODUCT_NAME,PRODUCT_CODE,STORE_CLUSTER,FORMAT_NAME,STORE_REGION,STORE_NAME,STORE_CODE,16/09/2024_SALES,...,07/10/2024_VOLUME,14/10/2024_VOLUME,21/10/2024_VOLUME,28/10/2024_VOLUME,04/11/2024_VOLUME,11/11/2024_VOLUME,18/11/2024_VOLUME,25/11/2024_VOLUME,02/12/2024_VOLUME,09/12/2024_VOLUME
0,WEEKLY,1600701 - Hot Food - Deli,TEGEL OVEN BAKED THIGHS HOT (S) KG 1,5269087_KG1,NW Urban Upscale,New World,NORTH,Stonefields New World,6614_4000,110.94,...,43.0,45.0,20.0,30.0,27.0,47.0,83.0,26.0,29.0,44.0
1,WEEKLY,1600701 - Hot Food - Deli,RW MEAL PACK BREAST EA EA 1,5137570_EA1,PNS Trolley Fillers,PAKnSAVE,SOUTH,PAK'nSAVE Moorhouse,500075_12,526.69,...,27.0,32.0,25.0,39.0,26.0,24.0,41.0,25.0,23.0,31.0


In [15]:
# Get all columns that contain '_SALES' in their name
sales_column = [col for col in waitoa_deli.columns if '_SALES' in col]

# Sum all sales columns to create total_sales
waitoa_deli['total_sales'] = waitoa_deli[sales_column].sum(axis=1)
# Drop all individual sales columns
waitoa_deli = waitoa_deli.drop(columns=sales_column)


## Do same for Volume
# Get all columns that contain '_SALES' in their name
volume_column = [col for col in waitoa_deli.columns if '_VOLUME' in col]

# Sum all sales columns to create total_sales
waitoa_deli['sales_volume'] = waitoa_deli[volume_column].sum(axis=1)

# Drop all individual sales columns
waitoa_deli = waitoa_deli.drop(columns=volume_column)

# Drop specified columns
columns_to_drop = ['AGG_TYPE', 'PRODUCT_GROUP', 'PRODUCT_CODE', 'STORE_CLUSTER','STORE_REGION']
waitoa_deli = waitoa_deli.drop(columns=columns_to_drop)

# Display first few rows to verify
waitoa_deli.head()


,PRODUCT_NAME,FORMAT_NAME,STORE_NAME,STORE_CODE,total_sales,sales_volume
0,TEGEL OVEN BAKED THIGHS HOT (S) KG 1,New World,Stonefields New World,6614_4000,2489.27,436.0
1,RW MEAL PACK BREAST EA EA 1,PAKnSAVE,PAK'nSAVE Moorhouse,500075_12,6527.13,387.0
2,TEGEL LOUISIANA NIBBLES HOT (S) KG 1,New World,Whangaparaoa New World,4116_4000,2892.90,445.0
3,PMS HT RST CHCKN SG AND ONN STFFD LRG EA 1,New World,Waipukurau New World,8043_4000,21191.21,1225.0
4,RW HOT CHIPS LARGE EA 1,New World,New World Blenheim,500079_11,5.29,1.0


In [16]:
## In the Product_name column, separate Waitoa from the rest of competitors

# Remove records containing "INGHAMS" in PRODUCT_NAME so we can separate Waitoa from the rest of competitors
waitoa_deli = waitoa_deli[~waitoa_deli['PRODUCT_NAME'].str.contains('INGHAMS', case=True, na=False)]

# Display first few rows to verify
waitoa_deli.head()

,PRODUCT_NAME,FORMAT_NAME,STORE_NAME,STORE_CODE,total_sales,sales_volume
0,TEGEL OVEN BAKED THIGHS HOT (S) KG 1,New World,Stonefields New World,6614_4000,2489.27,436.0
1,RW MEAL PACK BREAST EA EA 1,PAKnSAVE,PAK'nSAVE Moorhouse,500075_12,6527.13,387.0
2,TEGEL LOUISIANA NIBBLES HOT (S) KG 1,New World,Whangaparaoa New World,4116_4000,2892.90,445.0
3,PMS HT RST CHCKN SG AND ONN STFFD LRG EA 1,New World,Waipukurau New World,8043_4000,21191.21,1225.0
4,RW HOT CHIPS LARGE EA 1,New World,New World Blenheim,500079_11,5.29,1.0


In [ ]:
# Create the is_waitoa column
waitoa_deli['is_waitoa'] = waitoa_deli['PRODUCT_NAME'].str.contains('WAITOA', case=True, na=False)

# Create the pivot table
pivot_deli = waitoa_deli.pivot_table(
    index=['STORE_NAME', 'FORMAT_NAME'],
    values=['total_sales', 'sales_volume'],
    columns='is_waitoa',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Flatten the MultiIndex columns - modified to handle index columns correctly
pivot_deli.columns = [
    col[0] if col[1] == '' else  # For index columns
    f'{col[0]}_{"waitoa" if col[1] else "competitors"}'  # For value columns
    for col in pivot_deli.columns
]

# Reorder columns
column_order = [
    'STORE_NAME',
    'FORMAT_NAME',
    'total_sales_waitoa',
    'sales_volume_waitoa',
    'total_sales_competitors',
    'sales_volume_competitors'
]
pivot_deli = pivot_deli[column_order]

# Display the first few rows
pivot_deli

In [18]:
# Calculate share of sales and volume
pivot_deli['share_of_sales'] = pivot_deli['total_sales_waitoa'] / pivot_deli['total_sales_competitors']
pivot_deli['share_of_volume'] = pivot_deli['sales_volume_waitoa'] / pivot_deli['sales_volume_competitors']

# Display first few rows to verify
pivot_deli.head()


,STORE_NAME,FORMAT_NAME,total_sales_waitoa,sales_volume_waitoa,total_sales_competitors,sales_volume_competitors,share_of_sales,share_of_volume
0,Albany New World,New World,40365.62,2230.0,90322.25,7361.0,0.446907,0.302948
1,Albany PAKnSAVE,PAKnSAVE,39618.98,2636.0,224055.45,22385.0,0.176827,0.117757
2,Alderman Drive Henderson PAKnSAVE,PAKnSAVE,0.00,0.0,142196.89,12023.0,0.000000,0.000000
3,Aokautere New World,New World,0.00,0.0,40506.74,2948.0,0.000000,0.000000
4,Birkenhead New World,New World,58396.72,3224.0,75091.62,6467.0,0.777673,0.498531


In [19]:
pivot_deli_df = pivot_deli[['STORE_NAME', 'share_of_sales', 'share_of_volume']]

In [93]:
## Do same for Waito Deli
waitoa_butchery = pd.read_csv(r'G:\My Drive\together_projects\waitoa\fusion/together-dunnhumby-butchery-data.csv')
waitoa_butchery.head(2)


,AGG_TYPE,PRODUCT_GROUP,PRODUCT_NAME,PRODUCT_CODE,STORE_CLUSTER,FORMAT_NAME,STORE_REGION,STORE_NAME,STORE_CODE,09/09/2024_SALES,...,30/09/2024_VOLUME,07/10/2024_VOLUME,14/10/2024_VOLUME,21/10/2024_VOLUME,28/10/2024_VOLUME,04/11/2024_VOLUME,11/11/2024_VOLUME,18/11/2024_VOLUME,25/11/2024_VOLUME,02/12/2024_VOLUME
0,WEEKLY,15502 - Poultry - Butchery,NZ CHICKEN THIGHS BONE IN KG 1,5118844_KG1,NW Urban Upscale,New World,NORTH,Hobsonville New World,8232_4000,286.41,...,35.0,52.0,60.0,54.0,48.0,44.0,24.0,31.0,37.0,48.0
1,WEEKLY,Waitoa - 1550206 - Poultry Free Range - Butchery,WAITOA FR CHCKN THIGH FLT BNLS SKINLESS KG 1,5226622_KG1,<BLANK>,New World,CENTRE,Havelock North New World,8327_4000,NaN,...,NaN,1.0,1.0,2.0,1.0,6.0,NaN,NaN,13.0,18.0


In [22]:
# Get all columns that contain '_SALES' in their name
sales_column = [col for col in waitoa_butchery.columns if '_SALES' in col]

# Sum all sales columns to create total_sales
waitoa_butchery['total_sales'] = waitoa_butchery[sales_column].sum(axis=1)
# Drop all individual sales columns
waitoa_butchery = waitoa_butchery.drop(columns=sales_column)

# Get all columns that contain '_SALES' in their name
volume_column = [col for col in waitoa_butchery.columns if '_VOLUME' in col]

# Sum all sales columns to create total_sales
waitoa_butchery['sales_volume'] = waitoa_butchery[volume_column].sum(axis=1)

# Drop all individual sales columns
waitoa_butchery = waitoa_butchery.drop(columns=volume_column)

# Drop specified columns
columns_to_drop = ['AGG_TYPE', 'PRODUCT_GROUP', 'PRODUCT_CODE', 'STORE_CLUSTER','STORE_REGION']
waitoa_butchery = waitoa_butchery.drop(columns=columns_to_drop)

# Display first few rows to verify
waitoa_butchery.head()


,PRODUCT_NAME,FORMAT_NAME,STORE_NAME,STORE_CODE,total_sales,sales_volume
0,NZ CHICKEN THIGHS BONE IN KG 1,New World,Hobsonville New World,8232_4000,3808.05,636.0
1,WAITOA FR CHCKN THIGH FLT BNLS SKINLESS KG 1,New World,Havelock North New World,8327_4000,695.12,48.0
2,BIRD&BARROW CHCKN FREE RNGE WHOLE BAG KG KG 1,New World,New World St Martins,500015_11,9585.88,640.0
3,NZ CHICKEN LEGS KG 1,New World,Albany New World,6714_4000,8820.59,919.0
4,NZ CHICKEN TENDERLOINS KG 1,New World,Orewa New World,6235_4000,17575.00,2271.0


In [23]:
## In the Product_name column, separate Waitoa from the rest of competitors

# Remove records containing "INGHAMS" in PRODUCT_NAME so we can separate Waitoa from the rest of competitors
waitoa_butchery = waitoa_butchery[~waitoa_butchery['PRODUCT_NAME'].str.contains('INGHAMS', case=True, na=False)]

# Display first few rows to verify
waitoa_butchery.head()

,PRODUCT_NAME,FORMAT_NAME,STORE_NAME,STORE_CODE,total_sales,sales_volume
0,NZ CHICKEN THIGHS BONE IN KG 1,New World,Hobsonville New World,8232_4000,3808.05,636.0
1,WAITOA FR CHCKN THIGH FLT BNLS SKINLESS KG 1,New World,Havelock North New World,8327_4000,695.12,48.0
2,BIRD&BARROW CHCKN FREE RNGE WHOLE BAG KG KG 1,New World,New World St Martins,500015_11,9585.88,640.0
3,NZ CHICKEN LEGS KG 1,New World,Albany New World,6714_4000,8820.59,919.0
4,NZ CHICKEN TENDERLOINS KG 1,New World,Orewa New World,6235_4000,17575.00,2271.0


In [24]:
# Create the is_waitoa column
waitoa_butchery['is_waitoa'] = waitoa_butchery['PRODUCT_NAME'].str.contains('WAITOA', case=True, na=False)

# Create the pivot table
pivot_butchery = waitoa_butchery.pivot_table(
    index=['STORE_NAME', 'FORMAT_NAME'],
    values=['total_sales', 'sales_volume'],
    columns='is_waitoa',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Flatten the MultiIndex columns - modified to handle index columns correctly
pivot_butchery.columns = [
    col[0] if col[1] == '' else  # For index columns
    f'{col[0]}_{"waitoa" if col[1] else "competitors"}'  # For value columns
    for col in pivot_butchery.columns
]

# Reorder columns
column_order = [
    'STORE_NAME',
    'FORMAT_NAME',
    'total_sales_waitoa',
    'sales_volume_waitoa',
    'total_sales_competitors',
    'sales_volume_competitors'
]
pivot_butchery = pivot_butchery[column_order]

# Display the first few rows
pivot_butchery.head()

,STORE_NAME,FORMAT_NAME,total_sales_waitoa,sales_volume_waitoa,total_sales_competitors,sales_volume_competitors
0,Albany New World,New World,41809.10,2965.0,371353.74,37057.0
1,Albany PAKnSAVE,PAKnSAVE,15019.48,1726.0,1790108.28,188643.0
2,Alderman Drive Henderson PAKnSAVE,PAKnSAVE,4176.04,400.0,602755.95,64820.0
3,Aokautere New World,New World,3569.92,288.0,98593.03,11190.0
4,Birkenhead New World,New World,126402.32,8954.0,409495.69,44173.0


In [25]:
#pivot_butchery.to_csv('d:downloads/pivot_butchery.csv')

In [26]:
# Calculate share of sales and volume
pivot_butchery['share_of_sales'] = pivot_butchery['total_sales_waitoa'] / pivot_butchery['total_sales_competitors']
pivot_butchery['share_of_volume'] = pivot_butchery['sales_volume_waitoa'] / pivot_butchery['sales_volume_competitors']

# Display first few rows to verify
pivot_butchery.head()


,STORE_NAME,FORMAT_NAME,total_sales_waitoa,sales_volume_waitoa,total_sales_competitors,sales_volume_competitors,share_of_sales,share_of_volume
0,Albany New World,New World,41809.10,2965.0,371353.74,37057.0,0.112586,0.080012
1,Albany PAKnSAVE,PAKnSAVE,15019.48,1726.0,1790108.28,188643.0,0.008390,0.009150
2,Alderman Drive Henderson PAKnSAVE,PAKnSAVE,4176.04,400.0,602755.95,64820.0,0.006928,0.006171
3,Aokautere New World,New World,3569.92,288.0,98593.03,11190.0,0.036209,0.025737
4,Birkenhead New World,New World,126402.32,8954.0,409495.69,44173.0,0.308678,0.202703


In [27]:
pivot_butchery_df = pivot_butchery[['STORE_NAME', 'share_of_sales', 'share_of_volume']]

In [29]:
# Create a merged and harmonized DataFrame by concatenating all pivot dataframes
# Assign dataset identifiers to distinguish between them
pivot_frozen_df = pivot_frozen_df.copy()
pivot_frozen_df.loc[:, 'category'] = 'Frozen'

pivot_deli_df = pivot_deli_df.copy()
pivot_deli_df.loc[:, 'category'] = 'Deli'

pivot_butchery_df = pivot_butchery_df.copy()
pivot_butchery_df.loc[:, 'category'] = 'Butchery'


# Concatenate the dataframes
combined_df = pd.concat([pivot_frozen_df, pivot_deli_df, pivot_butchery_df], ignore_index=True)

# Group by STORE_NAME and calculate the mean for share_of_sales and share_of_volume
harmonized_df = combined_df.groupby('STORE_NAME')[['share_of_sales', 'share_of_volume']].mean().reset_index()

harmonized_df.sort_values('STORE_NAME', ascending=True)


,STORE_NAME,share_of_sales,share_of_volume
0,Albany New World,0.395408,0.369759
1,Albany PAKnSAVE,0.173329,0.173025
2,Alderman Drive Henderson PAKnSAVE,0.030194,0.037633
3,Aokautere New World,0.136579,0.154199
4,Birkenhead New World,0.569968,0.471202
...,...,...,...
201,Whanganui PAKnSAVE,0.101625,0.107656
202,Whangaparaoa New World,0.319822,0.308859
203,Whangarei PAKnSAVE,0.039643,0.046839
204,Whitby New World,0.546027,0.456729


In [89]:
import geopandas as gpd

# Read the NZ stores data
nz_supermarkets  = gpd.read_file(r'G:\My Drive\together_projects\waitoa\fusion\nz-supermarket-postcodes\supermarkets-postcodes.shp')
nz_supermarkets.head(2)

,RecordID,Brand,Address,latitude,longitude,Name,RegionName,SA22025_V1,sa2_code,SA22025__2,SA32025__2,region_cod,region,population,POSTCODE,geometry
0,1,Countdown,Cnr George Bolt Memorial & Ron Keats DriveAuck...,-37.00,174.79,Auckland Airport,Auckland,147900,147900,Auckland Airport,Mangere,2,Auckland,690,2022,"POLYGON ((174.80465 -36.93684, 174.80475 -36.9..."
1,2,Countdown,76 Quay StreetAuckland,-36.85,174.77,Auckland City,Auckland,134800,134800,Auckland-University,Auckland City Centre,2,Auckland,130,1010,"MULTIPOLYGON (((174.95443 -36.73184, 174.95402..."


In [32]:
# Drop geometry field and assign to new dataframe
#nz_supermarkets_sa2 = nz_supermarkets.drop(columns=['geometry'])
#nz_supermarkets_sa2.to_csv('d:/downloads/nz-supermarkets-sa2.csv')

In [94]:
# Read the waitoa stores geo locations file
waitoa_stores_geo = pd.read_csv(r'G:\My Drive\together_projects\waitoa\fusion/waitoa_stores_geo_locations.csv')
# Display first few rows
waitoa_stores_geo.head()


,STORE_NAME,FORMAT_NAME,store_lat,store_lon
0,Albany New World,New World,-36.843948,174.785721
1,Albany PAKnSAVE,PAKnSAVE,-36.843948,174.785721
2,Alderman Drive Henderson PAKnSAVE,PAKnSAVE,-36.881973,174.633139
3,Aokautere New World,New World,-39.644385,175.639197
4,Birkenhead New World,New World,-36.823985,174.793434


In [34]:
import geopandas as gpd
from shapely.geometry import Point

# Convert waitoa_stores_geo to a GeoDataFrame
waitoa_stores_geo['geometry'] = waitoa_stores_geo.apply(
    lambda row: Point(row['store_lon'], row['store_lat']), axis=1
)
waitoa_stores_gdf = gpd.GeoDataFrame(waitoa_stores_geo, geometry='geometry', crs=nz_supermarkets.crs)

# Perform spatial join
waitoa_match_gdf = gpd.sjoin(waitoa_stores_gdf, nz_supermarkets, how='inner', predicate='within')

# Extract matched rows
waitoa_match_df = waitoa_match_gdf.drop(columns=['index_right']).reset_index(drop=True)

# Identify unmatched rows from nz_supermarkets
matched_indices = waitoa_match_gdf['index_right'].unique()
unmatched_nz_supermarkets = nz_supermarkets.loc[~nz_supermarkets.index.isin(matched_indices)].reset_index(drop=True)



In [36]:
# Check for duplicates based on Address
duplicates = waitoa_match_df.duplicated(subset=['RecordID'], keep='first')

# Remove duplicates and keep first occurrence
waitoa_match_df = waitoa_match_df[~duplicates].reset_index(drop=True)

# Display first few rows of deduplicated dataframe
waitoa_match_df.head()

,STORE_NAME,FORMAT_NAME,store_lat,store_lon,geometry,RecordID,Brand,Address,latitude,longitude,Name,RegionName,SA22025_V1,sa2_code,SA22025__2,SA32025__2,region_cod,region,population,POSTCODE
0,Albany New World,New World,-36.843948,174.785721,POINT (174.78572 -36.84395),2,Countdown,76 Quay StreetAuckland,-36.85,174.77,Auckland City,Auckland,134800,134800,Auckland-University,Auckland City Centre,2,Auckland,130,1010
1,Albany New World,New World,-36.843948,174.785721,POINT (174.78572 -36.84395),3,Countdown,19-25 Victoria StreetAuckland,-36.85,174.76,Auckland Victoria Street West,Auckland,133400,133400,Hobson Ridge Central,Auckland City Centre,2,Auckland,4020,1010
2,Albany New World,New World,-36.843948,174.785721,POINT (174.78572 -36.84395),63,Countdown,"AMP Centre, Ground FloorLower Albert StreetAuc...",-36.84,174.77,Albert Street,Auckland,119301,119301,Inlet Waitemata Harbour,Inlets Auckland,2,Auckland,110,1010
3,Albany New World,New World,-36.843948,174.785721,POINT (174.78572 -36.84395),230,New World,"125 Queen Street, Auckland",-36.85,174.77,New World Metro Queen Street,Upper North Island,134800,134800,Auckland-University,Auckland City Centre,2,Auckland,130,1010
4,Alderman Drive Henderson PAKnSAVE,PAKnSAVE,-36.881973,174.633139,POINT (174.63314 -36.88197),16,Countdown,West City Shopping CentreHendersonAuckland,-36.88,174.63,Henderson,Auckland,127500,127500,Henderson Central,Henderson,2,Auckland,1420,0612


### Assign calculated scores to the matched waitoa stores

In [ ]:
waitoa_match_scores_df = pd.merge(harmonized_df, waitoa_match_df, on='STORE_NAME', how='inner')
waitoa_match_scores_df.head()

In [41]:
# Check for null values in each column
null_columns = waitoa_match_scores_df.isnull().sum()

# Display columns with null values (if any)
print("Columns with null values:")
print(null_columns[null_columns > 0])


Columns with null values:
RegionName    12
dtype: int64


In [42]:
# Check for duplicate rows in waitoa_match_scores_df
duplicates = waitoa_match_scores_df.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")

# Display the duplicate rows if any exist
if duplicates.sum() > 0:
    print("\nDuplicate rows:")
    print(waitoa_match_scores_df[duplicates])
    
    # Remove duplicates and keep first occurrence
    waitoa_match_scores_df = waitoa_match_scores_df.drop_duplicates().reset_index(drop=True)
    print(f"\nShape after removing duplicates: {waitoa_match_scores_df.shape}")


Number of duplicate rows: 0


In [46]:
# Step 1: Compute average share_of_sales and share_of_volume for each POSTCODE in waitoa_match_scores_df
postcode_averages = waitoa_match_scores_df.groupby('POSTCODE')[['share_of_sales', 'share_of_volume']].mean().reset_index()

# Step 2: Merge unmatched_nz_supermarkets with the computed postcode averages
unmatched_with_postcode = unmatched_nz_supermarkets.merge(
    postcode_averages,
    on='POSTCODE',
    how='left',
    suffixes=('', '_postcode_avg')  # To distinguish from overall averages
)

# Step 3: Identify rows in unmatched_nz_supermarkets with no POSTCODE match
overall_averages = waitoa_match_scores_df[['share_of_sales', 'share_of_volume']].mean()

# Step 4: Assign fallback overall averages for rows with no POSTCODE match
unmatched_with_postcode['share_of_sales'] = unmatched_with_postcode['share_of_sales'].fillna(overall_averages['share_of_sales'])
unmatched_with_postcode['share_of_volume'] = unmatched_with_postcode['share_of_volume'].fillna(overall_averages['share_of_volume'])

# Final DataFrame
final_unmatched_nz_supermarkets = unmatched_with_postcode.copy()


In [ ]:
#unmatched_with_postcode

In [48]:
# # Install required packages
# !pip install scikit-learn

In [49]:
from sklearn.preprocessing import MinMaxScaler

# Step 1: Concatenate waitoa_match_scores_df and final_unmatched_nz_supermarkets
combined_stores_df = pd.concat([waitoa_match_scores_df, final_unmatched_nz_supermarkets], ignore_index=True)

# Step 2: Aggregate population and share_of_volume at the POSTCODE level
postcode_aggregates = combined_stores_df.groupby('POSTCODE').agg(
    summed_population=('population', 'sum'),
    mean_share_of_volume=('share_of_volume', 'mean')
).reset_index()

# Step 3: Normalize summed population and mean share_of_volume
scaler = MinMaxScaler()
postcode_aggregates['normalized_population'] = scaler.fit_transform(postcode_aggregates[['summed_population']])
postcode_aggregates['normalized_share_of_volume'] = scaler.fit_transform(postcode_aggregates[['mean_share_of_volume']])

# Step 4: Compute ranking score for each POSTCODE
postcode_aggregates['ranking_score'] = (
    postcode_aggregates['normalized_population'] - postcode_aggregates['normalized_share_of_volume']
)

# Step 5: Rank POSTCODE groups by ranking score
postcode_aggregates = postcode_aggregates.sort_values(by='ranking_score', ascending=False).reset_index(drop=True)
postcode_aggregates['group_rank'] = postcode_aggregates.index + 1

# Step 6: Merge group ranks back to combined_stores_df
combined_stores_df = combined_stores_df.merge(
    postcode_aggregates[['POSTCODE', 'group_rank']],
    on='POSTCODE',
    how='left'
)

# Step 7: Sort final DataFrame by group rank and optionally within groups by ranking score
combined_stores_df = combined_stores_df.sort_values(by=['group_rank'], ascending=True).reset_index(drop=True)


In [ ]:
# Fill missing store_lat/store_lon values with latitude/longitude values and vice versa
combined_stores_df['store_lat'] = combined_stores_df.apply(lambda x: x['store_lat'] if pd.notnull(x['store_lat']) else x['latitude'], axis=1)
combined_stores_df['store_lon'] = combined_stores_df.apply(lambda x: x['store_lon'] if pd.notnull(x['store_lon']) else x['longitude'], axis=1)

# Fill any remaining nulls in latitude/longitude with store_lat/store_lon values
combined_stores_df['latitude'] = combined_stores_df.apply(lambda x: x['latitude'] if pd.notnull(x['latitude']) else x['store_lat'], axis=1)
combined_stores_df['longitude'] = combined_stores_df.apply(lambda x: x['longitude'] if pd.notnull(x['longitude']) else x['store_lon'], axis=1)

# Display the updated dataframe
#combined_stores_df

### Plot the geomap of 452 stores
 

In [ ]:
# !pip install geopy
# !pip install folium

In [55]:
# import libraries
from sklearn.cluster import KMeans
import numpy as np
from geopy.distance import great_circle
import folium

# Step 1: Apply KMeans clustering for exactly 100 clusters
def create_clusters_kmeans(df, n_clusters=100, min_radius_km=2):
    # Prepare coordinates for clustering
    coords = df[['latitude', 'longitude']].to_numpy()
    
    # Use KMeans for clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df['cluster'] = kmeans.fit_predict(coords)
    
    # Calculate cluster centroids
    centroids = kmeans.cluster_centers_
    df['centroid_lat'] = df['cluster'].map(lambda c: centroids[c][0])
    df['centroid_lon'] = df['cluster'].map(lambda c: centroids[c][1])
    
    return df

# Step 2: Calculate centroids and radii for clusters
def calculate_cluster_centroids_and_radii_kmeans(df, min_radius_km):
    cluster_info = []
    
    for cluster_id, group in df.groupby('cluster'):
        # Calculate centroid
        centroid_lat = group['centroid_lat'].iloc[0]
        centroid_lon = group['centroid_lon'].iloc[0]
        centroid = (centroid_lat, centroid_lon)
        
        # Calculate maximum radius
        max_radius = max(
            great_circle(centroid, (row['latitude'], row['longitude'])).kilometers
            for _, row in group.iterrows()
        ) if len(group) > 1 else min_radius_km  # Ensure a minimum radius for single-store clusters
        
        max_radius = max(max_radius, min_radius_km)  # Apply minimum radius
        
        # Store cluster info
        cluster_info.append({
            'cluster': cluster_id,
            'centroid_lat': centroid_lat,
            'centroid_lon': centroid_lon,
            'radius_km': max_radius
        })
    
    return pd.DataFrame(cluster_info)

# Step 3: Plot clusters on a Folium map
def plot_clusters_kmeans(df, cluster_info):
    # Initialize a Folium map
    nz_map = folium.Map(location=[-40.9006, 174.8860], zoom_start=5)
    
    # Plot individual stores
    for _, row in df.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"Store: {row['Brand']}<br>Cluster: {row['cluster']}<br>Population: {row['population']}",
            icon=folium.Icon(color='blue')
        ).add_to(nz_map)
    
    # Plot cluster circles
    for _, cluster_row in cluster_info.iterrows():
        folium.Circle(
            location=[cluster_row['centroid_lat'], cluster_row['centroid_lon']],
            radius=cluster_row['radius_km'] * 1000,  # Convert to meters
            color='green',
            fill=True,
            fill_opacity=0.3,
            popup=f"Cluster: {cluster_row['cluster']}<br>Radius: {cluster_row['radius_km']:.2f} km"
        ).add_to(nz_map)
    
    # Save the map
    nz_map.save('nz_store_clusters_kmeans.html')
    return nz_map

# Step 4: Process combined_stores_df
# Create clusters and calculate cluster details
combined_stores_df = create_clusters_kmeans(combined_stores_df, n_clusters=100, min_radius_km=3)
cluster_info = calculate_cluster_centroids_and_radii_kmeans(combined_stores_df, min_radius_km=3)

# # Step 5: Plot the clusters
# nz_map = plot_clusters_kmeans(combined_stores_df, cluster_info)

# # To display the map in an interactive environment or Jupyter Notebook
# nz_map
# Step 6: Remove specific clusters
def remove_clusters(df, cluster_info, clusters_to_remove):
    # Filter out the specified clusters
    df_filtered = df[~df['cluster'].isin(clusters_to_remove)]
    cluster_info_filtered = cluster_info[~cluster_info['cluster'].isin(clusters_to_remove)]
    return df_filtered, cluster_info_filtered

# List of clusters to remove
clusters_to_remove = [49, 80, 87]

# Filter out specified clusters
combined_stores_df_filtered, cluster_info_filtered = remove_clusters(
    combined_stores_df, cluster_info, clusters_to_remove
)

# Step 7: Plot the filtered clusters
nz_map_filtered = plot_clusters_kmeans(combined_stores_df_filtered, cluster_info_filtered)

# Display the updated map
nz_map_filtered


In [62]:
#nz_map_filtered.save('d:/downloads/nz_store_clusters_kmeans_filtered.html')

In [61]:
cluster_info_filtered 

,cluster,centroid_lat,centroid_lon,radius_km
0,0,-43.537143,172.672857,7.954720
1,1,-37.696058,176.139591,10.756189
2,2,-40.858571,175.062857,13.707920
3,3,-45.250000,169.390000,3.000000
4,4,-36.790833,174.751667,6.122263
...,...,...,...,...
95,95,-37.250000,174.730000,3.000000
96,96,-37.560000,175.920000,3.000000
97,97,-41.124286,174.860000,3.903381
98,98,-36.993000,174.878000,3.823630


In [72]:
combined_stores_df_filtered.head()


,STORE_NAME,share_of_sales,share_of_volume,FORMAT_NAME,store_lat,store_lon,geometry,RecordID,Brand,Address,...,SA22025__2,SA32025__2,region_cod,region,population,POSTCODE,group_rank,cluster,centroid_lat,centroid_lon
0,NaN,0.255525,0.216084,NaN,-37.74,176.12,"POLYGON ((176.15552 -37.69744, 176.15569 -37.6...",433,SuperValue,"83 Pyes Pa Road, Pyes Pa 3112",...,Pyes Pa North,Pyes Pa,4,Bay of Plenty,3980,3112,1,1,-37.696058,176.139591
1,NaN,0.255525,0.216084,NaN,-37.70,176.16,"POLYGON ((176.15552 -37.69744, 176.15569 -37.6...",382,PAK'nSAVE,"476 Cameron Road, Tauranga, 3110",...,Tauranga South,Tauranga South,4,Bay of Plenty,4240,3112,1,1,-37.696058,176.139591
2,NaN,0.255525,0.216084,NaN,-37.71,176.14,"POLYGON ((176.15552 -37.69744, 176.15569 -37.6...",262,New World,"948 Cameron Road, Tauranga",...,Gate Pa,Gate Pa,4,Bay of Plenty,4300,3112,1,1,-37.696058,176.139591
3,NaN,0.255525,0.216084,NaN,-37.70,176.16,"POLYGON ((176.15552 -37.69744, 176.15569 -37.6...",73,Countdown,618 Cameron Road Tauranga,...,Tauranga South,Tauranga South,4,Bay of Plenty,4240,3112,1,1,-37.696058,176.139591
4,NaN,0.255525,0.216084,NaN,-37.71,176.15,"POLYGON ((176.15552 -37.69744, 176.15569 -37.6...",68,Countdown,229-233 Fraser StreetFraser Cove Shopping Cent...,...,Fraser Cove,Tauranga South,4,Bay of Plenty,1310,3112,1,1,-37.696058,176.139591


In [63]:
## Get phyciall addresses for each cluster
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
import pandas as pd

# Initialize the geolocator
geolocator = Nominatim(user_agent="geoapi_exercise")

# Step 1: Define a function to get the address for a given latitude and longitude
def get_address(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True, language='en')
        return location.address if location else None
    except Exception as e:
        print(f"Error retrieving address for ({lat}, {lon}): {e}")
        return None

# Step 2: Get addresses for all clusters in cluster_info_filtered
def get_cluster_addresses(cluster_info):
    cluster_info['address'] = cluster_info.apply(
        lambda row: get_address(row['centroid_lat'], row['centroid_lon']),
        axis=1
    )
    return cluster_info

# Step 3: Handle clusters with missing addresses
def assign_nearest_address(cluster_info, combined_stores_df):
    for index, row in cluster_info.iterrows():
        if pd.isnull(row['address']):
            # Find the nearest store to the cluster centroid
            cluster_centroid = (row['centroid_lat'], row['centroid_lon'])
            nearest_store = combined_stores_df.iloc[
                combined_stores_df.apply(
                    lambda x: great_circle(cluster_centroid, (x['latitude'], x['longitude'])).kilometers,
                    axis=1
                ).idxmin()
            ]
            # Assign the address of the nearest store
            cluster_info.at[index, 'address'] = nearest_store['Address']
    return cluster_info

# Step 4: Process cluster_info_filtered to get addresses
# First, get the physical addresses for clusters
cluster_info_with_addresses = get_cluster_addresses(cluster_info_filtered)

# Next, assign nearest store addresses to clusters with missing addresses
cluster_info_with_complete_addresses = assign_nearest_address(cluster_info_with_addresses, combined_stores_df_filtered)



C:\Users\User\AppData\Local\Temp\ipykernel_35916\3547257871.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_info['address'] = cluster_info.apply(


In [65]:
cluster_info_with_complete_addresses

,cluster,centroid_lat,centroid_lon,radius_km,address
0,0,-43.537143,172.672857,7.954720,"2/12, Bonar Place, Woolston, Linwood-Central-H..."
1,1,-37.696058,176.139591,10.756189,"26, Paine Street, Judea, Tauranga, Tauranga Ci..."
2,2,-40.858571,175.062857,13.707920,"Ngarara, 66, Maurice Smith Way, Manu Park, Wai..."
3,3,-45.250000,169.390000,3.000000,"40, Bantry Street, Bridge Hill, Alexandra, Vin..."
4,4,-36.790833,174.751667,6.122263,"Northcote Road Shared Path, Hillcrest, Kaipāti..."
...,...,...,...,...,...
95,95,-37.250000,174.730000,3.000000,"Bowen Street, Waiuku, Franklin, Auckland, 2123..."
96,96,-37.560000,175.920000,3.000000,"Middlebrook Drive, Katikati, Western Bay of Pl..."
97,97,-41.124286,174.860000,3.903381,"Howden Place, Aotea, Porirua, Porirua City, We..."
98,98,-36.993000,174.878000,3.823630,"Kotuku Car Park, Putney Way, Manukau, Ōtara-Pa..."


In [66]:
# Check for any null addresses in cluster_info_with_complete_addresses
null_addresses = cluster_info_with_complete_addresses['address'].isnull().sum()
print(f"Number of null addresses: {null_addresses}")


Number of null addresses: 0


In [67]:
## Add Postcodes to Cluster Information
# Step 1: Group POSTCODES by cluster
postcode_groups = (
    combined_stores_df_filtered.groupby('cluster')['POSTCODE']
    .apply(lambda x: list(set(x)))  # Remove duplicates and format as list
    .reset_index(name='postcodes')  # Reset index and rename column
)

# Step 2: Merge grouped POSTCODES into cluster_info_with_complete_addresses
cluster_info_with_postcodes = cluster_info_with_complete_addresses.merge(
    postcode_groups,
    on='cluster',
    how='left'
)

# Display the updated DataFrame with postcodes attached


In [68]:
cluster_info_with_postcodes

,cluster,centroid_lat,centroid_lon,radius_km,address,postcodes
0,0,-43.537143,172.672857,7.954720,"2/12, Bonar Place, Woolston, Linwood-Central-H...","[8081, 8013, 8022, 8083, 8024, 8023, 8082, 806..."
1,1,-37.696058,176.139591,10.756189,"26, Paine Street, Judea, Tauranga, Tauranga Ci...","[3110, 3112, 3116, 3114]"
2,2,-40.858571,175.062857,13.707920,"Ngarara, 66, Maurice Smith Way, Manu Park, Wai...","[5036, 5032, 5512]"
3,3,-45.250000,169.390000,3.000000,"40, Bantry Street, Bridge Hill, Alexandra, Vin...",[9320]
4,4,-36.790833,174.751667,6.122263,"Northcote Road Shared Path, Hillcrest, Kaipāti...","[0627, 0620, 0624, 0629, 0626, 0622]"
...,...,...,...,...,...,...
92,95,-37.250000,174.730000,3.000000,"Bowen Street, Waiuku, Franklin, Auckland, 2123...",[2123]
93,96,-37.560000,175.920000,3.000000,"Middlebrook Drive, Katikati, Western Bay of Pl...",[3129]
94,97,-41.124286,174.860000,3.903381,"Howden Place, Aotea, Porirua, Porirua City, We...","[5024, 5026, 5022]"
95,98,-36.993000,174.878000,3.823630,"Kotuku Car Park, Putney Way, Manukau, Ōtara-Pa...","[2019, 2103, 2104, 2025, 2102]"


In [69]:
cluster_info_with_postcodes.to_csv('d:/downloads/cluster_info_with_postcodes.csv')

In [86]:
## Select columns for output for campaign 
columns_to_display = [
    'cluster',
    'sa2_code',
    'SA32025__2',
    'region',
    'POSTCODE',
    'store_lat',
    'store_lon', 
    'Brand',
    'Address',
    'share_of_sales',
    'share_of_volume',
    'population',
    'group_rank',
   ]

combined_stores_df_filtered[columns_to_display].to_csv('d:/downloads/all-stores-and-rank.csv')

In [ ]:
##### END #####